# Document Reviewer Agent using LangGraph, LangChain and Ollama

In [ ]:
%pip install langgraph langchain langchain_openai langchain_core langchain-community

In [ ]:
from langchain_ollama import ChatOllama
import os
import operator
from dataclasses import dataclass, field, fields
from typing_extensions import Annotated, Literal
import json
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables import RunnableConfig
from langchain_ollama import ChatOllama
from langgraph.graph import START, END, StateGraph
from langsmith import traceable
from tavily import TavilyClient
from typing import Any, Optional
from IPython.display import Image, display, Markdown

In [ ]:
llm=ChatOllama(model="deepseek-r1:latest", temperature=0)#initialize model-specify which one you want to use, pull from ollama before
llm_json_mode=ChatOllama(model="deepseek-r1:latest", temperature=0, format="json")#we generate structured outputs since this type of input needed for LangChain..
#temperature=0 means you want to generate deterministic outputs-same input same output

In [ ]:
#Here we define our state

@dataclass(kw_only=True)
class ReviewState:
    document_path: str = field(default=None) # The original doc
    sections: Annotated[list, operator.add] = field(default_factory=list)
    summaries: Annotated[list, operator.add] = field(default_factory=list) 
    reviews: Annotated[list, operator.add] = field(default_factory=list)
    final_report: str = field(default=None) # Final report

In [ ]:
#Reads the input file, splits it into paragraphs and returns them to be stored in state.sections
def load_document(state: ReviewState):
    with open(state.document_path, 'r') as f:
        text = f.read()
    sections = text.split("\n\n")  # paragraph-based splitting
    return {"sections": sections}
    
#For each section it prompts the LLM to summarize it and store the results in a list then return that list
def summarize_section(state: ReviewState):
    summaries = []
    for section in state.sections:
        prompt = f"Summarize the following section:\n\n{section}"
        result = llm.invoke([SystemMessage(content=prompt)])
        summaries.append(result.content.strip())
    return {"summaries": summaries}

#For each section it asks the LLM to critique it on clarity, grammar, and structure, stores that feedback
def review_sections(state: ReviewState):
    reviews = []
    for i, section in enumerate(state.sections):
        prompt = (
            f"Review this section of a document:\n\n{section}\n\n"
            f"Provide suggestions for clarity, grammar, and structure."
        )
        result = llm.invoke([SystemMessage(content=prompt)])
        reviews.append(result.content.strip())
    return {"reviews": reviews}

#For each section, show its summary and the review.
def compile_report(state: ReviewState):
    report = "## Document Review Report\n\n"
    for i, (summary, review) in enumerate(zip(state.summaries, state.reviews)):
        report += f"### Section {i+1}\n"
        report += f"**Summary:**\n{summary}\n\n"
        report += f"**Review:**\n{review}\n\n"
    return {"final_report": report}

In [ ]:
# Here we build the LangGraph
builder = StateGraph(ReviewState, output=ReviewState)
builder.set_entry_point("load_document")

builder.add_node("load_document", load_document)
builder.add_node("summarize_section", summarize_section)
builder.add_node("review_sections", review_sections)
builder.add_node("compile_report", compile_report)

builder.add_edge("load_document", "summarize_section")
builder.add_edge("summarize_section", "review_sections")
builder.add_edge("review_sections", "compile_report")
builder.add_edge("compile_report", END)

graph = builder.compile()
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
#Test-here we run the agent
state = ReviewState(document_path="research.txt")  # replace with your doc
result = graph.invoke(state)
print(result["final_report"])